### to delete the default network

In [ ]:
%%bash
. /home/stack/devstack/openrc admin admin
ROUTER_ID=$(neutron router-list | grep router1 | grep -v '+' | grep -v 'external_gateway' | awk '{ print $2 }')
PRIVATE_SUBNET_ID=$(neutron subnet-list | grep '| private-subnet' | awk '{print $2}')
PUBLIC_SUBNET_ID=$(neutron subnet-list | grep '| public-subnet' | awk '{print $2}')
neutron router-gateway-clear $ROUTER_ID

for i in `neutron port-list | grep -v '+' | grep -v 'fixed_ips' | awk '{print $2}'`;
do
  neutron port-update --device-owner None $i;
  neutron port-delete $i;
done;

neutron router-delete router1
neutron net-delete public
neutron net-delete private

### The code for Topology

In [3]:
%%bash
. /home/stack/devstack/openrc admin admin

neutron net-create --shared --provider:network_type flat --provider:physical_network flat --router:external my-public-network

neutron subnet-create --name my-public-subnet --gateway 10.10.10.1 --allocation-pool start=10.10.10.2,end=10.10.10.255 --dns-nameserver 8.8.8.8 my-public-network 10.10.0.0/16

neutron net-create my-private-network

neutron subnet-create my-private-network --name my-private-subnet --gateway 192.168.0.1 192.168.0.0/24

neutron router-create my-router

neutron router-interface-add my-router my-public-subnet

neutron router-interface-add my-router my-private-subnet

nova boot --flavor 1 --image cirros-0.3.4-x86_64-uec --security-groups default --availability-zone nova --nic net-name=my-private-network my-instance-01
nova boot --flavor 1 --image cirros-0.3.4-x86_64-uec --security-groups default --availability-zone nova --nic net-name=my-private-network my-instance-02


Created a new network:
+---------------------------+--------------------------------------+
| Field                     | Value                                |
+---------------------------+--------------------------------------+
| admin_state_up            | True                                 |
| availability_zone_hints   |                                      |
| availability_zones        |                                      |
| created_at                | 2016-11-16T22:40:26                  |
| description               |                                      |
| id                        | f66a96b3-455b-436d-a616-c6db2865671d |
| ipv4_address_scope        |                                      |
| ipv6_address_scope        |                                      |
| is_default                | False                                |
| mtu                       | 1500                                 |
| name                      | my-public-network                    |
| port_secu